In [1]:
from config import set_path, DATA_DIR

set_path()

import pandas as pd
from core.load_data.types import JSON_TYPE
from core.load_data import load_contributions

import os


In [2]:
data = load_contributions()

Incomplete read - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Too many requests - waiting 3 seconds
Incomplete read - waiting 3 seconds
Incomplete read - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Too many requests - waiting 3 seconds
Incomplete read - waiting 3 seconds
Finished - 204
Finished - 204
Too many requests - waiting 3 seconds
Finished - 204
Finished - 204


In [3]:
data.keys()

dict_keys(['Assistência Social', 'Cultura', 'Desenvolvimento Econômico e Trabalho', 'Direitos Humanos e Cidadania', 'Educação', 'Esporte e Lazer', 'Habitação', 'Infraestrutura Urbana e Obras', 'Inovação e Tecnologia', 'Meio Ambiente', 'Outros temas', 'Pessoa com Deficiência', 'Saúde', 'Segurança Urbana', 'Transportes e Mobilidade', 'Urbanismo e Licenciamento', 'Zeladoria Urbana e melhorias de bairro'])

Adicionando o tema às contribuições

In [4]:
all_data = []
for tema, contribs in data.items():
    for contrib in contribs:
        contrib['tema'] = tema
    all_data.extend(contribs)

In [5]:
all_data[0].keys()

dict_keys(['id', 'nomeMunicipe', 'canal', 'integra', 'sugestoes', 'tema'])

In [6]:
def parse_temas(contrib:JSON_TYPE)->list[str]:


    TEMA_GENERICO = 'Outros temas'
    tema_set =set(sugestao['tema']['categoria']['nome']
                   for sugestao in contrib['sugestoes']
                   )
    if TEMA_GENERICO in tema_set:
        tema_set.remove(TEMA_GENERICO)
    
    return list(tema_set)

Unpacking das sugestões para format flat (nome do municipe não interessa)

In [7]:
parsed_all_data: list[dict] = []
for contrib in all_data:
    parsed_data = {
        'id' : contrib['id'],
        'canal' : contrib['canal']['nome'],
        'texto' : contrib['integra'],
        'temas' : parse_temas(contrib)
    }

    parsed_all_data.append(parsed_data)

print(f'Total {len(parsed_all_data)} registros')
    

Total 40120 registros


In [8]:
parsed_all_data[:3]

[{'id': 1,
  'canal': 'Audiência Geral',
  'texto': 'Boa tarde a todas e todos, meu nome é Francisco, apelido Chicão, morador do Butantã na região do Monte Kemel, faço parte do movimento Parque Chácara do Jockey, sou integrante também do fórum Verde permanente de parques praças e áreas verdes da cidade de São Paulo.\n Acho, que a primeira manifestação dos coletivos todos e também das regiões, é a nossa preocupação com a falta de objetividade das metas, quer dizer as metas precisam ser melhor esclarecidas né. A gente não consegue identificar as metas e as regiões onde elas vão ser aplicadas, aonde elas podem sofrer o controle social da comunidade inclusive. Então essa é uma reivindicação para as etapas regionais, que a gente possa definir essas metas e melhorar a definição dessas metas e relacioná-las com as necessidades de cada região, de cada bairro, essa é uma preocupação que em a gente identificou na cidade.\nA segunda preocupação é com a questão da área de Meio Ambiente. Então a ge

Está com muitos registros repetidos, vou fazer o filtro

In [9]:
conteudos_unicos = set()
deduplicated_data = []
for registro in parsed_all_data:
    conteudo = registro['texto']
    if conteudo in conteudos_unicos:
        continue
    
    deduplicated_data.append(registro)
    conteudos_unicos.add(conteudo)

print(f'Total registros deduplicados: {len(deduplicated_data)}')

Total registros deduplicados: 2312


In [10]:
deduplicated_data[:3]

[{'id': 1,
  'canal': 'Audiência Geral',
  'texto': 'Boa tarde a todas e todos, meu nome é Francisco, apelido Chicão, morador do Butantã na região do Monte Kemel, faço parte do movimento Parque Chácara do Jockey, sou integrante também do fórum Verde permanente de parques praças e áreas verdes da cidade de São Paulo.\n Acho, que a primeira manifestação dos coletivos todos e também das regiões, é a nossa preocupação com a falta de objetividade das metas, quer dizer as metas precisam ser melhor esclarecidas né. A gente não consegue identificar as metas e as regiões onde elas vão ser aplicadas, aonde elas podem sofrer o controle social da comunidade inclusive. Então essa é uma reivindicação para as etapas regionais, que a gente possa definir essas metas e melhorar a definição dessas metas e relacioná-las com as necessidades de cada região, de cada bairro, essa é uma preocupação que em a gente identificou na cidade.\nA segunda preocupação é com a questão da área de Meio Ambiente. Então a ge

In [11]:
df = pd.DataFrame(deduplicated_data)

In [12]:
df.head()

,id,canal,texto,temas
0,1,Audiência Geral,"Boa tarde a todas e todos, meu nome é Francisc...",[Meio Ambiente]
1,2,Audiência Geral,"Oi Sou Mário Sérgio Bortolotto, por favor você...",[Urbanismo e Licenciamento]
2,3,Audiência Geral,Então boa tarde a todos e todas. Eu comprimen...,"[Zeladoria Urbana e melhorias de bairro, Desen..."
3,4,Audiência Geral,"Muito obrigada agradeço a oportunidade, eu tam...","[Zeladoria Urbana e melhorias de bairro, Desen..."
4,5,Audiência Geral,"Bom dia a todos, meu nome é Maria Luiza Kovals...",[Zeladoria Urbana e melhorias de bairro]


In [13]:
df.tail()

,id,canal,texto,temas
2307,3103,Audiência Regional - Jaçanã,Nós os moradores da Região observamos com o in...,"[Assistência Social, Zeladoria Urbana e melhor..."
2308,3104,Audiência Regional - Jaçanã,"Brigada, boa noite. Eu sou conselheira municip...","[Meio Ambiente, Habitação, Desenvolvimento Eco..."
2309,3105,Audiência Regional - Jaçanã,Primeiro eu sou Nelson Ferreira Filho. Na regi...,"[Pessoa com Deficiência, Desenvolvimento Econô..."
2310,3106,Audiência Regional - Jaçanã,"Olá, boa noite a todos. Meu nome é Lucas, eu f...","[Cultura, Habitação, Educação]"
2311,3107,Audiência Regional - Jaçanã,"Olá boa noite a todas,todos e todas. Quero sau...","[Habitação, Desenvolvimento Econômico e Trabal..."


In [14]:
df.sample(10)

,id,canal,texto,temas
1563,2299,Participe+,Protesista na área da Subprefeitura de Pinheir...,[Saúde]
1834,2574,Participe+,Faixas exclusivas de ônibus. Ação recomendada ...,[Transportes e Mobilidade]
1218,1915,Participe+,Meta 15 - Revisar - Que tipo de serviço? tem q...,[Assistência Social]
711,888,Participe+,Construção de UBS no Chácara Três Meninas dist...,[Saúde]
1986,2727,Participe+,CRIAR CASA DE CULTURA JD VILA FORMOSA REGIÃO D...,[Cultura]
188,189,Audiência Regional - Lapa,Oi boa noite a todos. É um prazer né participa...,[Cultura]
253,254,Audiência Temática 1 (SP Justa e Inclusiva),"Bom, obrigada pela oportunidade de estar aqui ...",[Pessoa com Deficiência]
747,1068,Participe+,Meta 60 - Manter - Publicar os locais dos novo...,[Meio Ambiente]
704,854,Participe+,Criação de uma Unidade de Referência à Saúde d...,[Saúde]
1949,2689,Participe+,Puericultura. Focar em consultas de puericultu...,[Saúde]


In [15]:
df.to_csv(os.path.join(DATA_DIR, 'dados_orginais_contribuicoes.csv'), sep=';')